In [2]:
import pdspy.modeling as modeling
import pdspy.dust as dust
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import os
import pickle
import glob
from time import time

In [3]:
# get all model data
with open ('../grid_metadata/corefull.txt', 'rb') as fp:
    core= pickle.load(fp)
with open ('../grid_metadata/cubefull.txt', 'rb') as fp:
    cube = np.array(pickle.load(fp))

In [4]:
dictionary=np.load("../grid_metadata/dictionary.npy") # rearranged parameters and parameters1 into 
                                        # dictionary to agree with order of grid/models and corefull

In [12]:
def run_yso_model(filename=None, Tstar=None, logL_star=None, \
        logM_disk=None, logR_disk=None, h_0=None, logR_in=None, gamma=None, \
        beta=None, logM_env=None, logR_env=None, f_cav=None, ksi=None, \
        loga_max=None, p=None, incl=None, photonfactor=1,photonfactortherm=1):

    # Set up the dust properties.

    dust_gen = dust.DustGenerator(dust.__path__[0]+"/data/diana_wice.hdf5")
    ddust = dust_gen(10.**loga_max / 1e4, p)
    env_dust_gen = dust.DustGenerator(dust.__path__[0]+\
            "/data/diana_wice.hdf5")
    edust = env_dust_gen(1.0e-4, 3.5)

    # Calculate alpha correctly.

    alpha = gamma + beta

    # Fix the scale height of the disk.

    h_0 *= (10.**logR_disk)**beta

    # Set up the model.

    model = modeling.YSOModel()
    model.add_star(luminosity=10.**logL_star, temperature=Tstar)
    model.set_spherical_grid(10.**logR_in, 10.**logR_env, 100, 101, 2, \
            code="radmc3d")
    model.add_pringle_disk(mass=10.**logM_disk, rmin=10.**logR_in, \
            rmax=10.**logR_disk, plrho=alpha, h0=h_0, plh=beta, dust=ddust)
    model.add_ulrich_envelope(mass=10.**logM_env, rmin=10.**logR_in, \
            rmax=10.**logR_env, cavpl=ksi, cavrfact=f_cav, dust=edust)
    model.grid.set_wavelength_grid(0.1,1.0e5,500,log=True)
    
    print("finished setting up model, now running thermal simulation")

    # Run the thermal simulation
    model.run_thermal(code="radmc3d", nphot=1e6*photonfactortherm, \
            modified_random_walk=True, verbose=False, setthreads=16, \
            timelimit=50000)
    
    print("finished running thermal simulation, now running SED")
    t2=time()
    
    # Run the SED.

    model.set_camera_wavelength(np.logspace(-1.,4.,500))
    model.run_sed(name="SED", nphot=1e5*photonfactor, loadlambda=True, incl=incl, \
            pa=0., dpc=140., code="radmc3d", camera_scatsrc_allfreq=True, \
            verbose=False, setthreads=16)
    print("finished running SED in %0.3fs" % (time() - t2))
    
    
    # Write out the file.
    model.write_yso("../grid/photonpack/"+filename)

In [5]:
t0 = time()
run_yso_model(filename="10x.hdf5", Tstar=dictionary[71]["Tstar"], logL_star=dictionary[71]["logL_star"], \
        logM_disk=dictionary[71]['logM_disk'], logR_disk=dictionary[71]['logR_disk'], h_0=dictionary[71]["h_0"], \
        logR_in=dictionary[71]["logR_in"], gamma=dictionary[71]["gamma"], \
        beta=dictionary[71]["beta"], logM_env=dictionary[71]["logM_env"], logR_env=dictionary[71]["logR_env"], \
        f_cav=dictionary[71]["f_cav"], ksi=dictionary[71]["ksi"], \
        loga_max=dictionary[71]["loga_max"], p=dictionary[71]["p"], incl=dictionary[71]["incl"],photonfactor=1e1)
print("10x photon run done in %0.3fs" % (time() - t0))

t1 = time()
run_yso_model(filename="100x.hdf5", Tstar=dictionary[71]["Tstar"], logL_star=dictionary[71]["logL_star"], \
        logM_disk=dictionary[71]['logM_disk'], logR_disk=dictionary[71]['logR_disk'], h_0=dictionary[71]["h_0"], \
        logR_in=dictionary[71]["logR_in"], gamma=dictionary[71]["gamma"], \
        beta=dictionary[71]["beta"], logM_env=dictionary[71]["logM_env"], logR_env=dictionary[71]["logR_env"], \
        f_cav=dictionary[71]["f_cav"], ksi=dictionary[71]["ksi"], \
        loga_max=dictionary[71]["loga_max"], p=dictionary[71]["p"], incl=dictionary[71]["incl"],photonfactor=1e2)
print("100x photon run done in %0.3fs" % (time() - t1))

finished setting up model, now running thermal simulation


KeyboardInterrupt: 

In [10]:
t0 = time()
run_yso_model(filename="46_10x.hdf5", Tstar=dictionary[46]["Tstar"], logL_star=dictionary[46]["logL_star"], \
        logM_disk=dictionary[46]['logM_disk'], logR_disk=dictionary[46]['logR_disk'], h_0=dictionary[46]["h_0"], \
        logR_in=dictionary[46]["logR_in"], gamma=dictionary[46]["gamma"], \
        beta=dictionary[46]["beta"], logM_env=dictionary[46]["logM_env"], logR_env=dictionary[46]["logR_env"], \
        f_cav=dictionary[46]["f_cav"], ksi=dictionary[46]["ksi"], \
        loga_max=dictionary[46]["loga_max"], p=dictionary[46]["p"], incl=dictionary[46]["incl"],photonfactor=1e1)
print("10x photon run done in %0.3fs" % (time() - t0))

t1 = time()
run_yso_model(filename="46_100x.hdf5", Tstar=dictionary[46]["Tstar"], logL_star=dictionary[46]["logL_star"], \
        logM_disk=dictionary[46]['logM_disk'], logR_disk=dictionary[46]['logR_disk'], h_0=dictionary[46]["h_0"], \
        logR_in=dictionary[46]["logR_in"], gamma=dictionary[46]["gamma"], \
        beta=dictionary[46]["beta"], logM_env=dictionary[46]["logM_env"], logR_env=dictionary[46]["logR_env"], \
        f_cav=dictionary[46]["f_cav"], ksi=dictionary[46]["ksi"], \
        loga_max=dictionary[46]["loga_max"], p=dictionary[46]["p"], incl=dictionary[46]["incl"],photonfactor=1e2)
print("100x photon run done in %0.3fs" % (time() - t1))

finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 3874.821s
10x photon run done in 4683.494s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 38797.161s
100x photon run done in 39618.291s


In [11]:
t0 = time()
run_yso_model(filename="68_10x.hdf5", Tstar=dictionary[68]["Tstar"], logL_star=dictionary[68]["logL_star"], \
        logM_disk=dictionary[68]['logM_disk'], logR_disk=dictionary[68]['logR_disk'], h_0=dictionary[68]["h_0"], \
        logR_in=dictionary[68]["logR_in"], gamma=dictionary[68]["gamma"], \
        beta=dictionary[68]["beta"], logM_env=dictionary[68]["logM_env"], logR_env=dictionary[68]["logR_env"], \
        f_cav=dictionary[68]["f_cav"], ksi=dictionary[68]["ksi"], \
        loga_max=dictionary[68]["loga_max"], p=dictionary[68]["p"], incl=dictionary[68]["incl"],photonfactor=1e1)
print("10x photon run done in %0.3fs" % (time() - t0))

t1 = time()
run_yso_model(filename="68_100x.hdf5", Tstar=dictionary[68]["Tstar"], logL_star=dictionary[68]["logL_star"], \
        logM_disk=dictionary[68]['logM_disk'], logR_disk=dictionary[68]['logR_disk'], h_0=dictionary[68]["h_0"], \
        logR_in=dictionary[68]["logR_in"], gamma=dictionary[68]["gamma"], \
        beta=dictionary[68]["beta"], logM_env=dictionary[68]["logM_env"], logR_env=dictionary[68]["logR_env"], \
        f_cav=dictionary[68]["f_cav"], ksi=dictionary[68]["ksi"], \
        loga_max=dictionary[68]["loga_max"], p=dictionary[68]["p"], incl=dictionary[68]["incl"],photonfactor=1e2)
print("100x photon run done in %0.3fs" % (time() - t1))

finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 3491.727s
10x photon run done in 3585.537s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 34871.821s
100x photon run done in 34962.781s


In [13]:
def run10(model):
    t0=time()
    run_yso_model(filename=str(model)+"_10x.hdf5", Tstar=dictionary[model]["Tstar"], logL_star=dictionary[model]["logL_star"], \
            logM_disk=dictionary[model]['logM_disk'], logR_disk=dictionary[model]['logR_disk'], h_0=dictionary[model]["h_0"], \
            logR_in=dictionary[model]["logR_in"], gamma=dictionary[model]["gamma"], \
            beta=dictionary[model]["beta"], logM_env=dictionary[model]["logM_env"], logR_env=dictionary[model]["logR_env"], \
            f_cav=dictionary[model]["f_cav"], ksi=dictionary[model]["ksi"], \
            loga_max=dictionary[model]["loga_max"], p=dictionary[model]["p"], incl=dictionary[model]["incl"],photonfactor=1e1)
    print("10x photon run done in %0.3fs" % (time() - t0))
    

In [30]:
run10(62)

finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 6730.086s
10x photon run done in 10423.864s


In [31]:
t0 = time()
run_yso_model(filename="0_10x.hdf5", Tstar=dictionary[0]["Tstar"], logL_star=dictionary[0]["logL_star"], \
        logM_disk=dictionary[0]['logM_disk'], logR_disk=dictionary[0]['logR_disk'], h_0=dictionary[0]["h_0"], \
        logR_in=dictionary[0]["logR_in"], gamma=dictionary[0]["gamma"], \
        beta=dictionary[0]["beta"], logM_env=dictionary[0]["logM_env"], logR_env=dictionary[0]["logR_env"], \
        f_cav=dictionary[0]["f_cav"], ksi=dictionary[0]["ksi"], \
        loga_max=dictionary[0]["loga_max"], p=dictionary[0]["p"], incl=dictionary[0]["incl"],photonfactor=1e1)
print("10x photon run done in %0.3fs" % (time() - t0))


finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 6960.514s
10x photon run done in 8348.020s


In [32]:
t0 = time()
run_yso_model(filename="131_10x.hdf5", Tstar=dictionary[131]["Tstar"], logL_star=dictionary[131]["logL_star"], \
        logM_disk=dictionary[131]['logM_disk'], logR_disk=dictionary[131]['logR_disk'], h_0=dictionary[131]["h_0"], \
        logR_in=dictionary[131]["logR_in"], gamma=dictionary[131]["gamma"], \
        beta=dictionary[131]["beta"], logM_env=dictionary[131]["logM_env"], logR_env=dictionary[131]["logR_env"], \
        f_cav=dictionary[131]["f_cav"], ksi=dictionary[131]["ksi"], \
        loga_max=dictionary[131]["loga_max"], p=dictionary[131]["p"], incl=dictionary[131]["incl"],photonfactor=1e1)
print("10x photon run done in %0.3fs" % (time() - t0))


finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 3664.370s
10x photon run done in 3783.014s


In [33]:
t0 = time()
run_yso_model(filename="210_10x.hdf5", Tstar=dictionary[210]["Tstar"], logL_star=dictionary[210]["logL_star"], \
        logM_disk=dictionary[210]['logM_disk'], logR_disk=dictionary[210]['logR_disk'], h_0=dictionary[210]["h_0"], \
        logR_in=dictionary[210]["logR_in"], gamma=dictionary[210]["gamma"], \
        beta=dictionary[210]["beta"], logM_env=dictionary[210]["logM_env"], logR_env=dictionary[210]["logR_env"], \
        f_cav=dictionary[210]["f_cav"], ksi=dictionary[210]["ksi"], \
        loga_max=dictionary[210]["loga_max"], p=dictionary[210]["p"], incl=dictionary[210]["incl"],photonfactor=1e1)
print("10x photon run done in %0.3fs" % (time() - t0))


finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 2331.415s
10x photon run done in 7383.195s


In [16]:
#noise=[   0,  131,  210,  317, 389,  700,  701,  741,  779,  796,  806,  987,\
#       1176, 1222, 1249, 1368, 1397, 1515, 1548, 1567, 1602, 1610,\
#       1618, 1805, 2040, 2084, 2167, 2250, 2267, 2326, 2386, 2442,\
#       2486, 2497, 2608, 2726, 2802, 2824, 2889, 3001, 3310,\
#       3338, 3507, 3545, 3578, 3606, 3615, 3664, 3717, 3733, 3737, 3777]

noise = [317,  377,  486,  489,  521,  659,\
         700,  701,  741,  779,  796,  806,  872, 1222, 1249, \
         1282, 1302, 1483, 1654, 1859, 1976, 1997, 2025, 2087, 2124, 2267,\
         2437, 2580, 2642, 2786, 2933, 2976, 3001, 3172, 3271, 3361, 3418,\
         3584, 3615, 3737]

todo=0
todo_list=[]
for i in range(len(noise)):
    if not str(noise[i])+"_10x.hdf5" in os.listdir("../grid/photonpack"):
        todo+=1
        todo_list.append(noise[i])
print(str(todo)+" models to run")

19 models to run


In [17]:
for i in range(len(todo_list)):
    run10(todo_list[i])

finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 2397.894s
10x photon run done in 31186.173s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 2540.046s
10x photon run done in 3652.406s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 3322.335s
10x photon run done in 9794.512s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 2313.081s
10x photon run done in 9423.624s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 5258.768s
10x photon run done in 16027.904s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running S

KeyboardInterrupt: 

In [ ]:
def run100(model):
    t0=time()
    run_yso_model(filename=str(model)+"_100x.hdf5", Tstar=dictionary[model]["Tstar"], logL_star=dictionary[model]["logL_star"], \
            logM_disk=dictionary[model]['logM_disk'], logR_disk=dictionary[model]['logR_disk'], h_0=dictionary[model]["h_0"], \
            logR_in=dictionary[model]["logR_in"], gamma=dictionary[model]["gamma"], \
            beta=dictionary[model]["beta"], logM_env=dictionary[model]["logM_env"], logR_env=dictionary[model]["logR_env"], \
            f_cav=dictionary[model]["f_cav"], ksi=dictionary[model]["ksi"], \
            loga_max=dictionary[model]["loga_max"], p=dictionary[model]["p"], incl=dictionary[model]["incl"],photonfactor=1e2)
    print("100x photon run done in %0.3fs" % (time() - t0))
    

In [ ]:
noise100=[1483, 1654, 2642, 2976]
todo100=0
todo_list100=[]
for i in range(len(noise100)):
    if not str(noise100[i])+"_100x.hdf5" in os.listdir("../grid/photonpack"):
        todo100+=1
        todo_list100.append(noise100[i])   
print(str(todo100)+" 100x models to run")

In [ ]:
for i in range(len(todo_list100)):
    run100(todo_list100[i])

In [44]:
def run10therm(model):
    t0=time()
    run_yso_model(filename=str(model)+"_10x_therm.hdf5", Tstar=dictionary[model]["Tstar"], logL_star=dictionary[model]["logL_star"], \
            logM_disk=dictionary[model]['logM_disk'], logR_disk=dictionary[model]['logR_disk'], h_0=dictionary[model]["h_0"], \
            logR_in=dictionary[model]["logR_in"], gamma=dictionary[model]["gamma"], \
            beta=dictionary[model]["beta"], logM_env=dictionary[model]["logM_env"], logR_env=dictionary[model]["logR_env"], \
            f_cav=dictionary[model]["f_cav"], ksi=dictionary[model]["ksi"], \
            loga_max=dictionary[model]["loga_max"], p=dictionary[model]["p"], incl=dictionary[model]["incl"],photonfactor=1e1\
            , photonfactortherm=1e1)
    print("10x photon run done in %0.3fs" % (time() - t0))
    

In [45]:
run10therm(0)
run10therm(62)

finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 8627.424s
10x photon run done in 25785.880s
finished setting up model, now running thermal simulation
finished running thermal simulation, now running SED
finished running SED in 5434.160s
10x photon run done in 49800.729s
